## Setup environment

In [29]:
!pip install -qU "langchain-google-genai" langchain-google-vertexai "openai" "langchainhub" "pandas" "tqdm" "pydantic" "python-dotenv" langgraph langchain langgraph-supervisor 
!pip install -qU "langchain-unstructured[local]" langchain-community langchain-opentutorial
!pip install -qU langsmith openevals openai

In [30]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
from langchain.chat_models import init_chat_model
MODEL_NAME = "gemini-2.0-flash"

In [32]:
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.prompts import ChatPromptTemplate
from langchain_opentutorial.graphs import visualize_graph

### Challenge info

In [44]:
CHALLENGE_ID = "02"
URL = None

root_dir = 'noitatiolpxe_bew'
ev_path = root_dir[::-1] + f'/EV-{CHALLENGE_ID}'

import json

with open(os.path.join(ev_path, 'challenge.json'), 'r') as f:
    challenge = json.load(f)
    URL = challenge['url']

URL = URL.replace('host.docker.internal', 'localhost') #FIXME: TEMP!
CHALLENGE_ID, URL

('02', 'http://localhost:3000')

In [45]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    ev_path + '/application',
    exclude=['Dockerfile', 'docker-entrypoint.sh'],
    loader_cls=TextLoader
)

docs = loader.load()
len(docs)

5

In [46]:
from langgraph.prebuilt import create_react_agent

# Define the prompt
executor_system_prompt = """
You are a helpful assistant.
"""

# Define the LLM
llm = init_chat_model(
    model=MODEL_NAME,
    temperature=0 #is this the correct syntax? 
)

In [47]:
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
test = python_repl.run(f'''
import requests

test = requests.get('{URL}')                
print(test.status_code)

''')
print(test)

from langchain_core.tools import Tool

repl_tool = Tool(
    name="python_repl",
    func=python_repl.run,
    description="A Python code REPL. Use this to automate tasks that require many repetitions. Input should be a valid python program. If you want to see the output of a value, you should print it out with `print(...)`.",
    return_direct=True,
)

200



In [48]:
# Create ReAct agent
agent_executor = create_react_agent(model=llm, tools=[repl_tool], prompt=executor_system_prompt)

In [63]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


# State definition
class PlanExecute(TypedDict):
    input: Annotated[str, "User's input"]
    plan: Annotated[List[str], "Current plan"]
    past_steps: Annotated[List[Tuple], operator.add]
    response: Annotated[str, "Final response"]

In [64]:
from pydantic import BaseModel, Field
from typing import List


# Define Plan model
class Plan(BaseModel):
    """Sorted steps to execute the plan"""

    steps: Annotated[List[str], "Different steps to follow, should be in sorted order"]

In [65]:
planner_system_prompt = ChatPromptTemplate.from_template("""
For the given objective, come up with a simple step by step plan.
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps.
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.
""")

planner_agent = planner_system_prompt | init_chat_model(
    model=MODEL_NAME,
).with_structured_output(Plan)


In [66]:
from typing import Union

# Define the prompt for re-planning
replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the following steps:
{past_steps}

Update your plan accordingly. If no more steps are needed return AN EMPTY PLAN. You do not need to include past steps in the plan.
"""
)


# Create the replanner
replanner = replanner_prompt | init_chat_model(
    model=MODEL_NAME
).with_structured_output(Plan)

In [67]:
test_prompt = """
Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the following steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan.
"""

test_input = replanner.invoke({
    'input': "Get the status code of the page",
    'plan': ["Get the status code of the page"],
    'past_steps': [("Get the status code of the page", 200)]
})

test_input

Plan(steps=[])

In [72]:
from langchain_core.output_parsers import StrOutputParser


# Generate and return a plan based on user input
def plan_step(state: PlanExecute):
    plan = planner_agent.invoke({"messages": [("user", state["input"])]})
    # Return the list of steps from the generated plan
    return {"plan": plan.steps}


# Use the agent executor to perform the specified task and return the result
def execute_step(state: PlanExecute):
    plan = state["plan"]
    # Convert the plan to a string, enumerating each step
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    # Format the current task for the agent
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing [step 1. {task}]."""
    # Use the agent executor to perform the task and get the result
    agent_response = agent_executor.invoke({"messages": [("user", task_formatted)]})
    # Return a dictionary containing the previous step and its result
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


# Update the plan or return the final response based on the results of the previous step
def replan_step(state: PlanExecute):
    output = replanner.invoke(state)

    # If more steps are needed
    next_plan = output.steps
    if len(next_plan) == 0:
        return {"response": "No more steps needed."}
    else:
        return {"plan": next_plan}


# A function that decides whether to end the agent's execution
def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return "final_report"
    else:
        return "execute"


final_report_prompt = ChatPromptTemplate.from_template(
    """You are given the objective and the previously done steps. Your task is to generate a final report in markdown format.
Final report should be written in professional tone.

Your objective was this:

{input}

Your previously done steps(question and answer pairs):

{past_steps}

Generate a final report in markdown format."""
)

final_report = (
    final_report_prompt
    | init_chat_model(model=MODEL_NAME, temperature=0)
    | StrOutputParser()
)


def generate_final_report(state: PlanExecute):
    past_steps = "\n\n".join(
        [
            f"Question: {past_step[0]}\n\nAnswer: {past_step[1]}\n\n####"
            for past_step in state["past_steps"]
        ]
    )
    response = final_report.invoke({"input": state["input"], "past_steps": past_steps})
    return {"response": response}

In [73]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver


# Create the workflow graph
workflow = StateGraph(PlanExecute)

# Define nodes
workflow.add_node("planner", plan_step)
workflow.add_node("execute", execute_step)
workflow.add_node("replan", replan_step)
workflow.add_node("final_report", generate_final_report)

# Define edges
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "execute")
workflow.add_edge("execute", "replan")
workflow.add_edge("final_report", END)

# Conditional edges: use should_end function to decide whether to stop
workflow.add_conditional_edges(
    "replan",
    should_end,
    {"execute": "execute", "final_report": "final_report"},
)

# Compile the graph
app = workflow.compile(checkpointer=MemorySaver())

In [74]:
from IPython.display import Image, display

app.get_graph().draw_ascii()

# Image(app.get_graph(xray=True).draw_mermaid_png(output_file_path="05-langgraph-plan-and-execute.png"))

'  +-----------+  \n  | __start__ |  \n  +-----------+  \n        *        \n        *        \n        *        \n  +---------+    \n  | planner |    \n  +---------+    \n        *        \n        *        \n        *        \n  +---------+    \n  | execute |    \n  +---------+    \n        *        \n        *        \n        *        \n   +--------+    \n   | replan |    \n   +--------+    \n        .        \n        .        \n        .        \n+--------------+ \n| final_report | \n+--------------+ \n        *        \n        *        \n        *        \n  +---------+    \n  | __end__ |    \n  +---------+    '

In [75]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(recursion_limit=50, configurable={"thread_id": "1"})
inputs = {"input": "Please explain about AI Agents."}

async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

{'plan': ['List the steps required to fulfill the objective.']}
{'past_steps': [('List the steps required to fulfill the objective.', 'Okay, I understand. To list the steps required to fulfill the objective, I need to first understand what the objective is. Please provide the objective. Once I have the objective, I can list the steps required to achieve it.\n')]}
{'plan': ['Define AI Agents.', 'Explain the key components of AI Agents.', 'Describe the different types of AI Agents.', 'Discuss the applications of AI Agents.', 'Summarize the explanation of AI Agents.']}
{'past_steps': [('Define AI Agents.', 'AI agents are autonomous entities, either software or hardware-based, that can perceive their environment through sensors, reason about their observations, and act upon that environment to achieve specific goals. They can be designed to be simple or complex, and they often exhibit characteristics such as learning, adaptation, and decision-making.\n')]}
{'plan': ['Explain the key compon

In [76]:
snapshot = app.get_state(config).values
print(snapshot["response"])

# AI Agents: An Overview

This report provides an overview of AI agents, covering their definition, key components, types, and applications.

## Definition

An AI agent is an autonomous entity, implemented as software or hardware, that perceives its environment through sensors and acts upon that environment through actuators to achieve a specific goal. The agent's intelligence lies in its ability to reason, learn, and adapt to changing conditions.

## Key Components

AI agents are characterized by the following key components:

*   **Perception:** This involves sensing and interpreting the environment. Agents use sensors (physical or virtual) to acquire data, preprocess it to remove noise and extract relevant features, and build a representation of their environment.
*   **Reasoning:** This is the agent's "thinking" process. It involves knowledge representation (how the agent stores information), inference (drawing conclusions), and decision-making (choosing the best course of action).

In [ ]:
from IPython.display import Markdown
Markdown(snapshot["response"])

# AI Agents: An Overview

This report provides an overview of AI agents, covering their definition, key components, types, and applications.

## Definition

An AI agent is an autonomous entity, implemented as software or hardware, that perceives its environment through sensors and acts upon that environment through actuators to achieve a specific goal. The agent's intelligence lies in its ability to reason, learn, and adapt to changing conditions.

## Key Components

AI agents are characterized by the following key components:

*   **Perception:** This involves sensing and interpreting the environment. Agents use sensors (physical or virtual) to acquire data, preprocess it to remove noise and extract relevant features, and build a representation of their environment.
*   **Reasoning:** This is the agent's "thinking" process. It involves knowledge representation (how the agent stores information), inference (drawing conclusions), and decision-making (choosing the best course of action).
*   **Action:** This is how the agent interacts with its environment. It involves using effectors (actuators) to perform actions, planning sequences of actions, and executing those actions. Feedback mechanisms allow the agent to learn from the consequences of its actions.
*   **Learning:** The ability to improve performance over time through experience.
*   **Autonomy:** The ability to operate independently without constant human intervention.

## Types of AI Agents

Different types of AI agents exist, each with varying degrees of complexity and capabilities:

*   **Simple Reflex Agents:** These agents react directly to percepts based on pre-defined rules. They have no memory of past states and operate based on a condition-action principle.
*   **Model-Based Reflex Agents:** These agents maintain an internal "model" of the environment to handle partially observable environments. This model is updated based on percepts and is used to infer hidden aspects of the current state.
*   **Goal-Based Agents:** These agents have a specific goal in mind and try to achieve it by planning and searching for sequences of actions. They use their model of the world to evaluate which actions will lead them closer to their goals.
*   **Utility-Based Agents:** These agents aim to maximize their "utility" or happiness, considering multiple factors and trade-offs. They use a utility function to assign a value to different states, allowing them to make decisions that maximize their overall well-being.

## Applications of AI Agents

AI agents have a wide range of applications across various domains:

*   **Robotics:** Controlling robots for tasks like manufacturing, exploration, and healthcare. AI-powered robots can assemble products, navigate unknown terrains, and assist in surgeries.
*   **Natural Language Processing (NLP):** Powering virtual assistants (e.g., Siri, Alexa), chatbots, and machine translation tools. AI agents can understand and respond to human language, providing information and completing tasks.
*   **Computer Vision:** Analyzing and interpreting images and videos for applications like self-driving cars, medical image analysis, and security surveillance. AI agents can identify objects, detect anomalies, and track movements.
*   **Game Playing:** Creating intelligent game-playing agents (e.g., chess, Go) that can compete with human players.
*   **Recommendation Systems:** Suggesting products, movies, or music based on user preferences.
*   **Autonomous Vehicles:** Driving cars and other vehicles without human intervention.
*   **Healthcare:** Assisting doctors with diagnosis, treatment planning, and patient monitoring.
*   **Finance:** Automating trading, fraud detection, and risk management.

## Conclusion

AI agents are a powerful technology with the potential to transform many aspects of our lives. Their ability to perceive, reason, and act autonomously makes them well-suited for a wide range of applications, from robotics and natural language processing to healthcare and finance. As AI technology continues to advance, we can expect to see even more innovative and impactful applications of AI agents in the future.
